In [1]:
import urllib.request
import json
import re
from urllib.error import HTTPError
from copy import deepcopy

In [2]:
sites = ['https://www.reuters.com/', 'https://www.afp.com/', 'http://headlinespot.com/', 'http://www.abyznewslinks.com/', 'http://www.lefigaro.fr/', 'https://www.theguardian.com']

In [16]:
dct = {}
for site in sites:
    print(site)
    try:
        html = urllib.request.urlopen(site).read().decode('utf-8')
    except UnicodeDecodeError:
        html = urllib.request.urlopen(site).read().decode('cp1252')
    except:
        continue
    adds = re.findall("<a href=\"(.+?)\"", html)
    dct[site] = adds

https://www.reuters.com/
https://www.afp.com/
http://headlinespot.com/
http://www.abyznewslinks.com/
http://www.lefigaro.fr/
https://www.theguardian.com


In [17]:
articles = {}
meta = {}
for key, value in dct.items():
    for elem in set(value[:20]):
        if elem.startswith('http'):
            url = elem
            if elem.count('/') < 5:
                continue
        else:
            if elem.count('/') < 2:
                continue
            if key.endswith('/') and elem.startswith('/'):
                url = key[:-1] + elem
            elif not key.endswith('/') and not elem.startswith('/'):
                url = key + '/' + elem
            else:
                url = key + elem
        if not url.startswith(key):
            continue
        print(url)
        try:
            html = urllib.request.urlopen(url).read().decode('utf-8')
        except:
            continue
        text = ' '.join(re.findall('<p.*?>(.*?)</p>', html, re.DOTALL))
        text = re.sub('<.*?>', '', text, 0, re.DOTALL)
        text = re.sub('\n\n', '\n', text)
        text_meta = ' '.join(re.findall("<meta(.*?)/?>", html, re.DOTALL))
        if key in articles:
            articles[key].append(text)
        else:
            articles[key] = [text]
        if key in meta:
            meta[key].append(text_meta)
        else:
            meta[key] = [text_meta]

https://www.reuters.com/subjects/aerospace-and-defense
https://www.reuters.com/finance/markets/us
https://www.reuters.com/finance/deals
https://www.reuters.com/markets/bonds
https://www.reuters.com/finance/markets/asia
https://www.reuters.com/subjects/banks
https://www.reuters.com/finance/markets/europe
https://www.reuters.com/finance/summits
https://www.reuters.com/finance/markets
https://www.reuters.com/subjects/autos
https://www.reuters.com/finance/global-market-data
https://www.reuters.com/markets/stocks
https://www.afp.com/fr/lagence/propos/lafp-en-chiffres
https://www.afp.com/fr/propos-de-lafp
https://www.afp.com/fr/rejoignez-nous
https://www.afp.com/fr/lagence/propos/lafp-en-images
https://www.afp.com/fr/produits-services
https://www.afp.com/fr/lagence/propos/lafp-dans-le-monde
https://www.afp.com/fr/lagence/prix-et-recompenses
https://www.afp.com/fr/lagence/communiques-de-presse
https://www.afp.com/fr/rapport_d_activite_2017
https://www.afp.com/fr/lagence/charte
https://www.afp

In [18]:
unique = set()
for key, value in meta.items():
    for i, elem in enumerate(value):
        elem = re.sub('\s+', ' ', elem, 0, re.DOTALL).strip()
        value[i] = elem
        tmp = re.findall('([^ \"]*?)=\"?[^\"]*?\"? ', elem)
        unique |= set(tmp)

In [19]:
unique

{'affiliate-data',
 'app-argument',
 'charset',
 'content',
 'http-equiv',
 'initial-scale',
 'itemprop',
 'name',
 'property',
 'user-scalable'}

In [20]:
meta_full = deepcopy(meta)

In [21]:
meta = deepcopy(meta_full)

In [22]:
for key, value in meta.items():
    for i, w in enumerate(value):
        w = re.sub('name=Description', 'name="Description"', w, 0)
        tmp = re.findall('([^ ]*?)=\"([^\"]*?)\"' , w)
        dct = {}
        fl1 = 0
        fl2 = 0
        for elem in tmp:
            if elem[0] in ['charset', 'initial-scale', 'maximum-scale', 'minimum-scale', 'shrink-to-fit', 'user-scalable', 'data-dynamic', 'data-react-helmet', 'affiliate-data']:
                dct[elem[0]] = elem[1]
            elif elem[0] in ['http-equiv', 'name', 'property', 'itemprop']:
                fl1 = 1
                if fl2 == 1:
                    if elem[1] in dct:
                        continue
                    dct[elem[1]] = q
                    fl1 = 0
                    fl2 = 0
                else:
                    q = elem[1]
            elif elem[0] in ['content']:
                fl2 = 1
                if fl1 == 1:
                    if q in dct:
                        continue
                    dct[q] = elem[1]
                    fl1 = 0
                    fl2 = 0
                else:
                    q = elem[1]
                    fl2 = 1
        value[i] = dct

In [23]:
meta.keys()

dict_keys(['https://www.reuters.com/', 'https://www.afp.com/', 'http://headlinespot.com/', 'https://www.theguardian.com'])

In [32]:
unique = set()
for key, value in meta.items():
    for elem in value:
        for inner_key, inner_value in elem.items():
            unique |= {inner_key}

In [36]:
with open('articles.json', 'w', encoding='utf-8') as f:
    json.dump(articles, f)

In [37]:
with open('meta.json', 'w', encoding='utf-8') as f:
    json.dump(meta, f)

In [2]:
dct = {}
for media, info in zip(['https://www.reuters.com/', 'https://www.afp.com/', 'http://headlinespot.com/', 'https://www.theguardian.com'], [['Reuters', 'English', 'UK'], ['Agence France-Presse', 'French', 'France'], ['Headline Spot', 'English', 'USA'], ['The Guardian', 'English', 'UK']]):
    dct[media] = info

In [4]:
with open('info.json', 'w', encoding='utf-8') as f:
    json.dump(dct, f)

In [3]:
dct

{'https://www.reuters.com/': ['Reuters', 'English', 'UK'],
 'https://www.afp.com/': ['Agence France-Presse', 'French', 'France'],
 'http://headlinespot.com/': ['Headline Spot', 'English', 'USA'],
 'https://www.theguardian.com': ['The Guardian', 'English', 'UK']}